In [1]:
import os
import scanpy as sc
import scvi
import json
from sklearn.model_selection import train_test_split
import numpy as np

/home/katwre/miniconda3/envs/fl-course-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Define data paths
data_dir = "data_input"
os.makedirs(data_dir, exist_ok=True)

pancreas_adata_path = os.path.join(data_dir, "pancreas_full.h5ad")
train_path = os.path.join(data_dir, "pancreas_train.h5ad")
valid_path = os.path.join(data_dir, "pancreas_valid.h5ad")
test_path  = os.path.join(data_dir, "pancreas_test.h5ad")

# Download if missing, otherwise load from local file
pancreas_adata = sc.read(
    pancreas_adata_path,
    backup_url="https://figshare.com/ndownloader/files/24539828",
)

# Split dataset by technology: keep smartseq2/celseq2 as held-out test
query_mask = pancreas_adata.obs["tech"].isin(["smartseq2", "celseq2"]).to_numpy()
pancreas_no_test = pancreas_adata[~query_mask].copy()
pancreas_test    = pancreas_adata[ query_mask].copy()

# 80/20 train/valid split on the remaining data, stratified by technology
y = pancreas_no_test.obs["tech"].astype("category")
indices = np.arange(pancreas_no_test.n_obs)

idx_train, idx_valid = train_test_split(
    indices,
    test_size=0.20,
    train_size=0.80,
    random_state=42,
    shuffle=True,
    stratify=y  # stratify by technology
)

pancreas_train = pancreas_no_test[idx_train].copy()
pancreas_valid = pancreas_no_test[idx_valid].copy()

# Save splits
pancreas_train.write(train_path)
pancreas_valid.write(valid_path)
pancreas_test.write(test_path)

print(
    f"Train: {pancreas_train.n_obs} cells | "
    f"Valid: {pancreas_valid.n_obs} cells | "
    f"Test: {pancreas_test.n_obs} cells"
)

# Print counts per technology
print("\nCells per technology:")
for name, ad in [("Train", pancreas_train),
                 ("Valid", pancreas_valid),
                 ("Test", pancreas_test)]:
    counts = ad.obs["tech"].value_counts().sort_index()
    print(f"\n{name} split:")
    for tech, n in counts.items():
        print(f"  {tech}: {n}")

# --- Cleanup: delete the original full dataset file ---
del pancreas_adata  # drop reference to ensure no open handle
try:
    if os.path.exists(pancreas_adata_path):
        os.remove(pancreas_adata_path)
        print(f"Deleted '{pancreas_adata_path}'")
except Exception as e:
    print(f"[WARN] Could not delete '{pancreas_adata_path}': {e}")

100%|██████████| 301M/301M [00:21<00:00, 14.5MB/s] 


Train: 9362 cells | Valid: 2341 cells | Test: 4679 cells

Cells per technology:

Train split:
  celseq: 803
  fluidigmc1: 510
  inDrop1: 1550
  inDrop2: 1379
  inDrop3: 2884
  inDrop4: 1042
  smarter: 1194

Valid split:
  celseq: 201
  fluidigmc1: 128
  inDrop1: 387
  inDrop2: 345
  inDrop3: 721
  inDrop4: 261
  smarter: 298

Test split:
  celseq2: 2285
  smartseq2: 2394
Deleted 'data_input/pancreas_full.h5ad'


In [ ]:
# Utility to load HVG list
def load_hvg_list(hvg_list_path):
    with open(hvg_list_path) as f:
        return json.load(f)

hvg_list = load_hvg_list("data_input/hvg_list.json")

# Restrict to HVG genes
pancreas_train = pancreas_train[:, hvg_list].copy()




pancreas_train
AnnData object with n_obs × n_vars = 9362 × 2000
    obs: 'tech', 'celltype', 'size_factors'
    layers: 'counts'


In [17]:
print("pancreas_train")
print(pancreas_train.X)
print(pancreas_train.X.shape)

print("\n pancreas_train obs")
print(pancreas_train.obs.head())
print(pancreas_train.obs.shape)


print("\n pancreas_train.obs.columns")
print(pancreas_train.obs.columns)

print("\npancreas_train.var.head()")
print(pancreas_train.var.head())

pancreas_train
[[0.         0.         0.         ... 0.         0.86844474 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(9362, 2000)

 pancreas_train obs
                                   tech celltype  size_factors  _scvi_batch  \
3rd-C86_S85                  fluidigmc1    delta      5.060723            1   
human3_lib4.final_cell_0804     inDrop3    alpha      0.010361            4   
human3_lib4.final_cell_0815     inDrop3     beta      0.011553            4   
Sample_163                      smarter     beta      1.000000            6   
human3_lib1.final_cell_0737     inDrop3    alpha      0.014493            4   

                             _sc

## Train of scVI model


In [7]:
scvi.model.SCVI.setup_anndata(pancreas_train, batch_key="tech", layer="counts")

scvi_ref = scvi.model.SCVI(
    pancreas_train,
    use_layer_norm="both",
    use_batch_norm="none",
    encode_covariates=True,
    dropout_rate=0.2,
    n_layers=2,
)
scvi_ref.train(max_epochs=50)

/home/katwre/miniconda3/envs/fl-course-env/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:63: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/katwre/miniconda3/envs/fl-course-env/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 1/50:   0%|          | 0/50 [00:00<?, ?it/s]

/home/katwre/miniconda3/envs/fl-course-env/lib/python3.11/site-packages/scvi/module/_vae.py:573: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)
/home/katwre/miniconda3/envs/fl-course-env/lib/python3.11/site-packages/scvi/module/_vae.py:573: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)


Epoch 2/50:   2%|▏         | 1/50 [00:02<01:57,  2.40s/it, v_num=1, train_loss_step=962, train_loss_epoch=1.31e+3]

/home/katwre/miniconda3/envs/fl-course-env/lib/python3.11/site-packages/scvi/module/_vae.py:573: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)
/home/katwre/miniconda3/envs/fl-course-env/lib/python3.11/site-packages/scvi/module/_vae.py:573: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)


...

Epoch 50/50:  98%|█████████▊| 49/50 [02:01<00:02,  2.41s/it, v_num=1, train_loss_step=883, train_loss_epoch=802]

/home/katwre/miniconda3/envs/fl-course-env/lib/python3.11/site-packages/scvi/module/_vae.py:573: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)
/home/katwre/miniconda3/envs/fl-course-env/lib/python3.11/site-packages/scvi/module/_vae.py:573: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs[MODULE_KEYS.PX_KEY].log_prob(x).sum(-1)


Epoch 50/50: 100%|██████████| 50/50 [02:04<00:00,  2.43s/it, v_num=1, train_loss_step=788, train_loss_epoch=802]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 50/50: 100%|██████████| 50/50 [02:04<00:00,  2.49s/it, v_num=1, train_loss_step=788, train_loss_epoch=802]


In [8]:
scvi_ref.save("model_centralized", overwrite=True)